In [1]:
# 1)
# instalamos 'ffmpeg'
!sudo apt update
!sudo apt install -y ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,472 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,275 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packa

In [2]:
# comprobamos que se ha instalado correctamente con las librerias
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [3]:
# 2)
# vamos a crear una clase llamada convert_color, para convertir un color RGB a YUV y viceversa
# utilizaremos la fórmula de las diapositivas de teoria

import subprocess

class convert_color:
    def rgb_to_yuv(self, r, g, b):
        y = 0.257 * r + 0.504 * g + 0.098 * b + 16
        u = -0.148 * r - 0.291 * g + 0.439 * b + 128
        v = 0.439 * r - 0.368 * g - 0.071 * b + 128
        return y, u, v

    def yuv_to_rgb(self, y, u, v):
        r = 1.164 * (y - 16) + 2.018 * (u - 128)
        g = 1.164 * (y - 16) - 0.813 * (v - 128) - 0.391 * (u - 128)
        b = 1.164 * (y - 16) + 1.596 * (v - 128)
        # nos aseguramos de que los valores de RGB estén dentro del rango [0, 255]
        r = min(max(0, r), 255)
        g = min(max(0, g), 255)
        b = min(max(0, b), 255)
        return int(r), int(g), int(b)

    # este método redimensiona una imagen y reduce la calidad utilizando FFmpeg
    def reduce_image(self, input_path, output_path, width=640, height=480, quality=5):
        command = [
            'ffmpeg', '-i', input_path,
            '-vf', f'scale={width}:{height}',
            '-qscale:v', str(quality),
            output_path
        ]
        # ejecutamos el comando de FFmpeg
        subprocess.run(command, check=True)
        print(f"Imagen ha redimensionada y guardada en {output_path}")

    # este método lee los bytes de una imagen
    def serpentine(self, file_path):
        with open(file_path, 'rb') as f:
            # primeor leemos los bytes del archivo
            bytes_data = f.read()

            # ahora convertimos los bytes en una lista de filas (cada fila 256 bytes)
            row_length = 256
            rows = [bytes_data[i:i + row_length] for i in range(0, len(bytes_data), row_length)]

            # ahora procesamos las filas de la forma serpentine
            serpentine_bytes = []
            for idx, row in enumerate(rows):
                if idx % 2 == 0:
                    # fila par: de izquierda a derecha
                    serpentine_bytes.extend(row)
                else:
                    # fila impar: de derecha a izquierda
                    serpentine_bytes.extend(row[::-1])

            print("Bytes serpentine form:")
            print(serpentine_bytes[:256])  # imprimimos solo los primeros 256 bytes

    # este método convierte una imagen en blanco y negro aplicando la compresión más fuerte posible
    def convert_to_bw(self, input_path, output_path):
        # este comando convierte la imagen a blanco y negro comprimiendo con la máxima calidad
        command = [
            'ffmpeg', '-i', input_path,
            '-vf', 'format=gray',  # convertimos la imagen a blanco y negro
            '-qscale:v', '31',     # aplicamos la máxima compresión (peor calidad)
            output_path
        ]
        # ejecutamos el comando de FFmpeg
        subprocess.run(command, check=True)
        print(f"La imagen ha sido convertida a blanco y negro guardada en {output_path}")

    # este método aplica el run-lenght encoding a una secuencia de bytes
    def run_length_encoding(self, byte_series):
        rle = []
        n = len(byte_series)
        i = 0

        while i < n:
            count = 1
            while i + 1 < n and byte_series[i] == byte_series[i + 1]:
                i += 1
                count += 1
            rle.append((byte_series[i], count))
            i += 1

        return rle

In [4]:
# probamos la clase
# RGB a YUV
r, g, b = 120, 65, 210
y, u, v = convert_color().rgb_to_yuv(r, g, b)
print(f"RGB ({r}, {g}, {b}) -> YUV ({y:.2f}, {u:.2f}, {v:.2f})")

# YUV a RGB
r_back, g_back, b_back = convert_color().yuv_to_rgb(y, u, v)
print(f"YUV ({y:.2f}, {u:.2f}, {v:.2f}) -> RGB ({r_back}, {g_back}, {b_back})")

RGB (120, 65, 210) -> YUV (100.18, 183.51, 141.85)
YUV (100.18, 183.51, 141.85) -> RGB (210, 65, 120)


In [6]:
# 3)
# añadimos un nuevo método a la clase convert_color llamado reduce_image para reducir las imagenes a una calidad inferior
# con el siguiente código probamos la clase:
imagen = 'campus_upf.jpg'
convert_color().reduce_image(imagen, 'output.jpg', width=320, height=240, quality=5)

Imagen ha redimensionada y guardada en output.jpg


In [7]:
# 4)
# añadimos un nuevo método a la clase convert_color llamado serpentine para leer los bytes del archivo utilizando la forma serpentine
# con el siguiente código probamos la clase:
convert_color().serpentine(imagen)

Bytes serpentine form:
[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0, 1, 2, 0, 0, 1, 0, 1, 0, 0, 255, 219, 0, 67, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 4, 3, 2, 2, 2, 2, 5, 4, 4, 3, 4, 6, 5, 6, 6, 6, 5, 6, 6, 6, 7, 9, 8, 6, 7, 9, 7, 6, 6, 8, 11, 8, 9, 10, 10, 10, 10, 10, 6, 8, 11, 12, 11, 10, 12, 9, 10, 10, 10, 255, 219, 0, 67, 1, 2, 2, 2, 2, 2, 2, 5, 3, 3, 5, 10, 7, 6, 7, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 255, 194, 0, 17, 8, 2, 72, 3, 32, 3, 1, 34, 0, 2, 17, 1, 3, 17, 1, 255, 196, 0, 29, 0, 0, 0, 7, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 0, 8, 9, 255, 196, 0, 27, 1, 0, 2, 3, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 255, 218, 0, 12, 3, 1, 0, 2, 16, 3, 16, 0, 0, 1, 245, 144, 31, 187, 24]


In [8]:
# 5)
# añadimos un nuevo método a la clase convert_color llamado convert_to_bw para convertir una imagen a blanco y negro aplicando la compresión máxima, es decir la peor calidad
# con el siguiente código probamos la clase:
convert_color().convert_to_bw(imagen, 'output_bw.jpg')

La imagen ha sido convertida a blanco y negro guardada en output_bw.jpg


In [9]:
# 5)
# añadimos un nuevo método a la clase convert_color llamado run_length_encoding
# con el siguiente código probamos la clase:
byte_series = [17, 8, 54, 0, 0, 0, 97, 5, 16, 0, 45, 23, 0, 0, 0, 0, 0, 3, 67, 0, 0, 8]
rle_result = convert_color().run_length_encoding(byte_series)
print("Run-Lenght Encoding (RLE):")
print(rle_result)

Run-Lenght Encoding (RLE):
[(17, 1), (8, 1), (54, 1), (0, 3), (97, 1), (5, 1), (16, 1), (0, 1), (45, 1), (23, 1), (0, 5), (3, 1), (67, 1), (0, 2), (8, 1)]


In [10]:
# 6)
# creamos la clase DCT

import numpy as np
from scipy.fftpack import dct, idct

class DCTConverter:
    def __init__(self):
        """Inicializa la clase DCTConverter"""
        pass

    # este metodo aplica la DCT a una secuencia de datos
    def apply_dct(self, data):
        if isinstance(data, list):
            data = np.array(data)  # convertimos a un array numpy si es una lista
        return dct(data, norm='ortho')

    # este metodo aplica la IDCT a los datos transformados
    def apply_idct(self, transformed_data):
        if isinstance(transformed_data, list):
            transformed_data = np.array(transformed_data)  # convertimos a un array numpy si es una lista
        return idct(transformed_data, norm='ortho')

    # este metodo aplica la DCT y elimina frecuencias por debajo de un umbral
    def compress_using_dct(self, data, threshold=0.1):
        # Primero, aplicamos DCT
        transformed_data = self.apply_dct(data)

        # aqui eliminamos las frecuencias de baja amplitud
        transformed_data[np.abs(transformed_data) < threshold] = 0

        return transformed_data

    # este metodo aplica la IDCT a los datos comprimidos para recuperar la secuencia original
    def decompress_using_dct(self, transformed_data):
        return self.apply_idct(transformed_data)

In [11]:
# probamos la clase

data = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# aplicamos la DCT a los datos
transformed_data = DCTConverter().apply_dct(data)
print("Datos obtenidos después de aplicar la DCT:")
print(transformed_data)

# aplicamos IDCT a los datos transformados
recovered_data = DCTConverter().apply_idct(transformed_data)
print("\nDatos obtenidos después de aplicar IDCT:")
print(recovered_data)

# comprimimos los datos utilizando la DCT (eliminando frecuencias por debajo de un umbral)
compressed_data = DCTConverter().compress_using_dct(data, threshold=10)
print("\nDatos comprimidos usando DCT:")
print(compressed_data)

# recuperamos los datos comprimidos
decompressed_data = DCTConverter().decompress_using_dct(compressed_data)
print("\nDatos después de descomprimir utilizando DCT:")
print(decompressed_data)

Datos obtenidos después de aplicar la DCT:
[ 1.73925271e+02 -9.02485113e+01  0.00000000e+00 -9.66656903e+00
  5.14153554e-15 -3.16227766e+00  3.73554423e-15 -1.27870393e+00
  0.00000000e+00 -3.58573004e-01]

Datos obtenidos después de aplicar IDCT:
[ 10.  20.  30.  40.  50.  60.  70.  80.  90. 100.]

Datos comprimidos usando DCT:
[173.92527131 -90.24851126   0.           0.           0.
   0.           0.           0.           0.           0.        ]

Datos después de descomprimir utilizando DCT:
[15.13654181 19.03865484 26.4609149  36.67677944 48.68624849 61.31375151
 73.32322056 83.5390851  90.96134516 94.86345819]


In [12]:
pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.3 MB/s eta 0:00:00


In [13]:
# 7)
# nos piden lo mismo que en el ejercicio anterior, pero esta vez con DWT
import pywt

class DWTConverter:
    def __init__(self, wavelet='haar', level=1):
        self.wavelet = wavelet
        self.level = level

    def apply_dwt(self, data):
        if isinstance(data, list):
            data = np.array(data)
        coeffs = pywt.wavedec(data, self.wavelet, level=self.level)
        return coeffs

    def apply_idwt(self, coeffs):
        # aplicamos la IDWT utilizando pywt
        recovered_data = pywt.waverec(coeffs, self.wavelet)
        return recovered_data

    def compress_using_dwt(self, data, threshold=0.1):
        # aplicamos la DWT
        coeffs = self.apply_dwt(data)

        # aqui eliminamos las frecuencias de baja amplitud
        for i in range(len(coeffs)):
            coeffs[i] = np.where(np.abs(coeffs[i]) < threshold, 0, coeffs[i])

        return coeffs

    def decompress_using_dwt(self, coeffs):
        # recuperamos los datos aplicando la IDWT
        return self.apply_idwt(coeffs)

In [14]:
# probamos la clase

data = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# aplicamos la DWT a los datos
transformed_data = DWTConverter().apply_dwt(data)
print("Datos obtenidos después de aplicar la DWT:")
print(transformed_data)

# aplicamos IDWT a los datos transformados
recovered_data = DWTConverter().apply_idwt(transformed_data)
print("\nDatos obtenidos después de aplicar IDCT:")
print(recovered_data)

# comprimimos los datos utilizando la DWT (eliminando frecuencias por debajo de un umbral)
compressed_data = DWTConverter().compress_using_dwt(data, threshold=10)
print("\nDatos comprimidos usando DWT:")
print(compressed_data)

# recuperamos los datos comprimidos
decompressed_data = DWTConverter().decompress_using_dwt(compressed_data)
print("\nDatos después de descomprimir utilizando DWT:")
print(decompressed_data)

Datos obtenidos después de aplicar la DWT:
[array([ 21.21320344,  49.49747468,  77.78174593, 106.06601718,
       134.35028843]), array([-7.07106781, -7.07106781, -7.07106781, -7.07106781, -7.07106781])]

Datos obtenidos después de aplicar IDCT:
[ 10.  20.  30.  40.  50.  60.  70.  80.  90. 100.]

Datos comprimidos usando DWT:
[array([ 21.21320344,  49.49747468,  77.78174593, 106.06601718,
       134.35028843]), array([0., 0., 0., 0., 0.])]

Datos después de descomprimir utilizando DWT:
[15. 15. 35. 35. 55. 55. 75. 75. 95. 95.]


In [16]:
# 8)
# Use any AI (YES, you can NOW, you lazy!) to create UNIT TESTS to your code, for each method and class

import unittest
import numpy as np
from scipy.fftpack import dct, idct


class TestConvertColor(unittest.TestCase):
    def test_rgb_to_yuv(self):
        cc = convert_color()
        y, u, v = cc.rgb_to_yuv(120, 65, 210)
        self.assertAlmostEqual(y, 89.81, places=2)
        self.assertAlmostEqual(u, 162.13, places=2)
        self.assertAlmostEqual(v, 146.73, places=2)

    def test_yuv_to_rgb(self):
        cc = convert_color()
        r, g, b = cc.yuv_to_rgb(89.81, 162.13, 146.73)
        self.assertEqual((r, g, b), (120, 65, 210))

    def test_run_length_encoding(self):
        cc = convert_color()
        byte_series = [1, 1, 2, 3, 3, 3, 4]
        rle = cc.run_length_encoding(byte_series)
        self.assertEqual(rle, [(1, 2), (2, 1), (3, 3), (4, 1)])


class TestDCTConverter(unittest.TestCase):
    def test_apply_dct(self):
        dct_converter = DCTConverter()
        data = [10, 20, 30, 40, 50]
        transformed = dct_converter.apply_dct(data)
        self.assertTrue(isinstance(transformed, np.ndarray))

    def test_apply_idct(self):
        dct_converter = DCTConverter()
        data = [10, 20, 30, 40, 50]
        transformed = dct_converter.apply_dct(data)
        recovered = dct_converter.apply_idct(transformed)
        np.testing.assert_almost_equal(data, recovered, decimal=5)

    def test_compress_using_dct(self):
        dct_converter = DCTConverter()
        data = [10, 20, 30, 40, 50]
        compressed = dct_converter.compress_using_dct(data, threshold=10)
        self.assertTrue(all(compressed >= 0))


class TestDWTConverter(unittest.TestCase):
    def test_apply_dwt(self):
        dwt_converter = DWTConverter()
        data = [10, 20, 30, 40, 50]
        transformed = dwt_converter.apply_dwt(data)
        self.assertTrue(isinstance(transformed, list))

    def test_apply_idwt(self):
        dwt_converter = DWTConverter()
        data = [10, 20, 30, 40, 50]
        transformed = dwt_converter.apply_dwt(data)
        recovered = dwt_converter.apply_idwt(transformed)
        np.testing.assert_almost_equal(data, recovered, decimal=5)

    def test_compress_using_dwt(self):
        dwt_converter = DWTConverter()
        data = [10, 20, 30, 40, 50]
        compressed = dwt_converter.compress_using_dwt(data, threshold=10)
        for coeffs in compressed:
            self.assertTrue(all(np.abs(coeffs) >= 0))

if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)


test_rgb_to_yuv (__main__.TestConvertColor) ... FAIL
test_run_length_encoding (__main__.TestConvertColor) ... ok
test_yuv_to_rgb (__main__.TestConvertColor) ... FAIL
test_apply_dct (__main__.TestDCTConverter) ... ok
test_apply_idct (__main__.TestDCTConverter) ... ok
test_compress_using_dct (__main__.TestDCTConverter) ... FAIL
test_apply_dwt (__main__.TestDWTConverter) ... ok
test_apply_idwt (__main__.TestDWTConverter) ... FAIL
test_compress_using_dwt (__main__.TestDWTConverter) ... ok

FAIL: test_rgb_to_yuv (__main__.TestConvertColor)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-16-facccd175e09>", line 13, in test_rgb_to_yuv
    self.assertAlmostEqual(y, 89.81, places=2)
AssertionError: 100.17999999999999 != 89.81 within 2 places (10.36999999999999 difference)

FAIL: test_yuv_to_rgb (__main__.TestConvertColor)
----------------------------------------------------------------------
Traceback (most recent 